# DSO106 MachineLearn L3 - Hands On

# Part I
# Create a decision tree model of the Titanic dataset that predicts survival from seaborn.

## Load in data

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [68]:
Titanic = sns.load_dataset('titanic')

In [69]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Data Wrangling

In [70]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### Drop the missing data

In [71]:
Titanic.dropna(inplace = True)
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     182 non-null    int64   
 1   pclass       182 non-null    int64   
 2   sex          182 non-null    object  
 3   age          182 non-null    float64 
 4   sibsp        182 non-null    int64   
 5   parch        182 non-null    int64   
 6   fare         182 non-null    float64 
 7   embarked     182 non-null    object  
 8   class        182 non-null    category
 9   who          182 non-null    object  
 10  adult_male   182 non-null    bool    
 11  deck         182 non-null    category
 12  embark_town  182 non-null    object  
 13  alive        182 non-null    object  
 14  alone        182 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 18.2+ KB


### Drop redundant variables

### 'sex', 'who', and 'adult_male' all appeared to be gender related

In [72]:
Titanic.drop(['who', 'adult_male'], axis = 1, inplace = True)
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,yes,False
3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,yes,False
6,0,1,male,54.0,0,0,51.8625,S,First,E,Southampton,no,True
10,1,3,female,4.0,1,1,16.7000,S,Third,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,C,Southampton,yes,True


### Now 'sex' is only gender related variable

### 'pclass' and 'class' also seem related

In [73]:
Titanic.pclass.value_counts()

1    157
2     15
3     10
Name: pclass, dtype: int64

In [74]:
Titanic['class'].value_counts()

First     157
Second     15
Third      10
Name: class, dtype: int64

## The .class.value_counts would not run -- kept saying invalid syntax

In [75]:
Titanic[Titanic['pclass']== 2]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alive,alone
21,1,2,male,34.0,0,0,13.0000,S,Second,D,Southampton,yes,True
66,1,2,female,29.0,0,0,10.5000,S,Second,F,Southampton,yes,True
123,1,2,female,32.5,0,0,13.0000,S,Second,E,Southampton,yes,True
148,0,2,male,36.5,0,2,26.0000,S,Second,F,Southampton,no,False
183,1,2,male,1.0,2,1,39.0000,S,Second,F,Southampton,yes,False
193,1,2,male,3.0,1,1,26.0000,S,Second,F,Southampton,yes,False
292,0,2,male,36.0,0,0,12.8750,C,Second,D,Cherbourg,no,True
327,1,2,female,36.0,0,0,13.0000,S,Second,D,Southampton,yes,True
340,1,2,male,2.0,1,1,26.0000,S,Second,F,Southampton,yes,False
345,1,2,female,24.0,0,0,13.0000,S,Second,F,Southampton,yes,True


### 'class' should be dropped

In [76]:
Titanic.drop('class', axis = 1, inplace = True)

In [77]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck,embark_town,alive,alone
1,1,1,female,38.0,1,0,71.2833,C,C,Cherbourg,yes,False
3,1,1,female,35.0,1,0,53.1000,S,C,Southampton,yes,False
6,0,1,male,54.0,0,0,51.8625,S,E,Southampton,no,True
10,1,3,female,4.0,1,1,16.7000,S,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,C,Southampton,yes,True


## 'survived' and 'alive' seem the same

In [78]:
Titanic.survived.value_counts()

1    123
0     59
Name: survived, dtype: int64

In [79]:
Titanic.alive.value_counts()

yes    123
no      59
Name: alive, dtype: int64

In [80]:
Titanic.drop('alive', axis =1, inplace = True)
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck,embark_town,alone
1,1,1,female,38.0,1,0,71.2833,C,C,Cherbourg,False
3,1,1,female,35.0,1,0,53.1000,S,C,Southampton,False
6,0,1,male,54.0,0,0,51.8625,S,E,Southampton,True
10,1,3,female,4.0,1,1,16.7000,S,G,Southampton,False
11,1,1,female,58.0,0,0,26.5500,S,C,Southampton,True


## Drop the variables not related to y

In [81]:
Titanic.drop('alone', axis = 1, inplace = True)

In [82]:
Titanic.drop('embark_town', axis = 1, inplace = True)

In [83]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
1,1,1,female,38.0,1,0,71.2833,C,C
3,1,1,female,35.0,1,0,53.1000,S,C
6,0,1,male,54.0,0,0,51.8625,S,E
10,1,3,female,4.0,1,1,16.7000,S,G
11,1,1,female,58.0,0,0,26.5500,S,C


## Convert everything to integers

In [84]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  182 non-null    int64   
 1   pclass    182 non-null    int64   
 2   sex       182 non-null    object  
 3   age       182 non-null    float64 
 4   sibsp     182 non-null    int64   
 5   parch     182 non-null    int64   
 6   fare      182 non-null    float64 
 7   embarked  182 non-null    object  
 8   deck      182 non-null    category
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 13.3+ KB


In [85]:
Titanic.age = Titanic.age.astype(int)

In [86]:
Titanic.fare = Titanic.fare.astype(int)

In [87]:
Titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  182 non-null    int64   
 1   pclass    182 non-null    int64   
 2   sex       182 non-null    object  
 3   age       182 non-null    int32   
 4   sibsp     182 non-null    int64   
 5   parch     182 non-null    int64   
 6   fare      182 non-null    int32   
 7   embarked  182 non-null    object  
 8   deck      182 non-null    category
dtypes: category(1), int32(2), int64(4), object(2)
memory usage: 11.9+ KB


## Recode strings

In [88]:
Titanic.deck.value_counts()

C    51
B    43
D    31
E    30
A    12
F    11
G     4
Name: deck, dtype: int64

In [89]:
def Deck (series):
    if series == 'A':
        return 0
    if series == 'B':
        return 1
    if series == 'C':
        return 2
    if series == 'D':
        return 3
    if series == 'E':
        return 4
    if series == 'F':
        return 5
    if series == 'G':
        return 6

Titanic['deckR'] = Titanic['deck'].apply(Deck)
Titanic.drop('deck', axis = 1, inplace = True)
Titanic.deckR = Titanic.deckR.astype(int)

In [90]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deckR
1,1,1,female,38,1,0,71,C,2
3,1,1,female,35,1,0,53,S,2
6,0,1,male,54,0,0,51,S,4
10,1,3,female,4,1,1,16,S,6
11,1,1,female,58,0,0,26,S,2


In [91]:
Titanic.embarked.value_counts()

S    115
C     65
Q      2
Name: embarked, dtype: int64

In [92]:
def Embark (series):
    if series == 'S':
        return 0
    if series == 'C':
        return 1
    if series == 'Q':
        return 2
    
Titanic['embarkedR'] = Titanic['embarked'].apply(Embark)
Titanic.drop('embarked', axis = 1, inplace = True)
Titanic.embarkedR = Titanic.embarkedR.astype(int)

In [93]:
Titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,deckR,embarkedR
1,1,1,female,38,1,0,71,2,1
3,1,1,female,35,1,0,53,2,0
6,0,1,male,54,0,0,51,4,0
10,1,3,female,4,1,1,16,6,0
11,1,1,female,58,0,0,26,2,0


In [94]:
Titanic.sex.value_counts()

male      94
female    88
Name: sex, dtype: int64

In [95]:
def Gender (series):
    if series == 'male':
        return 0
    if series == 'female':
        return 1
    
Titanic['sexR'] = Titanic['sex'].apply(Gender)
Titanic.drop('sex', axis = 1, inplace = True)
Titanic.sexR = Titanic.sexR.astype(int)

In [96]:
Titanic.head()

,survived,pclass,age,sibsp,parch,fare,deckR,embarkedR,sexR
1,1,1,38,1,0,71,2,1,1
3,1,1,35,1,0,53,2,0,1
6,0,1,54,0,0,51,4,0,0
10,1,3,4,1,1,16,6,0,1
11,1,1,58,0,0,26,2,0,1


# Define x and y

In [97]:
y = Titanic.survived
y.head()

1     1
3     1
6     0
10    1
11    1
Name: survived, dtype: int64

In [98]:
x = Titanic.drop('survived', axis = 1)
x.head()

,pclass,age,sibsp,parch,fare,deckR,embarkedR,sexR
1,1,38,1,0,71,2,1,1
3,1,35,1,0,53,2,0,1
6,1,54,0,0,51,4,0,0
10,3,4,1,1,16,6,0,1
11,1,58,0,0,26,2,0,1


## Train Test Split

In [99]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 76)

# Decision Tree Time

In [100]:
decisionTree = DecisionTreeClassifier(random_state = 76)
decisionTree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=76)

In [101]:
treePredictions = decisionTree.predict(x_test)
print(confusion_matrix(y_test, treePredictions))
print(classification_report(y_test, treePredictions))

[[14  5]
 [ 9 27]]
              precision    recall  f1-score   support

           0       0.61      0.74      0.67        19
           1       0.84      0.75      0.79        36

    accuracy                           0.75        55
   macro avg       0.73      0.74      0.73        55
weighted avg       0.76      0.75      0.75        55



# Model is 76% precise. Predicitng death properly 61% of time and survived properly 84% of the time

# Random Forest Time

In [102]:
forest = RandomForestClassifier(n_estimators = 500, random_state = 76)
forest.fit(x_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=76)

In [103]:
forestPredictions = forest.predict(x_test)
print(confusion_matrix(y_test, forestPredictions))
print(classification_report(y_test, forestPredictions))

[[14  5]
 [ 6 30]]
              precision    recall  f1-score   support

           0       0.70      0.74      0.72        19
           1       0.86      0.83      0.85        36

    accuracy                           0.80        55
   macro avg       0.78      0.79      0.78        55
weighted avg       0.80      0.80      0.80        55



## This model is a better fit with 80% precision. The Random forest Model is the more accurate one.